![](https://raw.githubusercontent.com/unm-escape/escape2022/main/header2.png)

<h1 style="text-align:center; color:#3333ff;">Intro to ObsPy</h1>
<br>
<div style="text-align:center; font-size:16px">
    Earth and Planetary Sciences,<br>
    University of New Mexico<br>
    <br>
    August 15, 2022
</div>

# [ObsPy](https://docs.obspy.org/)
---
**ObsPy Tutorial Outline:**
1. Downloading station inventory
2. Downloading earthquake catalog
3. Downloading earthquake seismograms<br>
4. Removing instrument response<br>
5. Writing waveform data to disk<br>
6. Reading waveform data from disk
7. Miscellaneous<br>
---
**Dependencies:** Obspy, Matplotlib, Numpy

---

## 1. Downloading station inventory

ObsPy can retrieve seismic data from many different online databases globally. These databases, known as "clients" in the ObsPy ecosystem are coordinated by the International Federation of Digital Seismograph Networks ([FDSN](https://docs.obspy.org/packages/obspy.clients.fdsn.html)).

In order to retrieve station metadata, earthquake catalogs or seismograms with obsPy, a client object must be initialized. In this tutorial we will use the IRIS webservice.

More information on basic FDSN client usage can be found [here](https://docs.obspy.org/packages/obspy.clients.fdsn.html#basic-fdsn-client-usage).

---

Let's begin by importing fdsn client package.

In [ ]:
from obspy.clients.fdsn import Client as FDSN_Client

Now we specify that we want data from the IRIS web service.

In [ ]:
client = FDSN_Client("IRIS")

ObsPy uses the [UTCDateTime](https://docs.obspy.org/packages/autogen/obspy.core.utcdatetime.UTCDateTime.html) class to work with time. So we have to import the UTCDatetime package too.

In [ ]:
from obspy import UTCDateTime

How do we decide which stations to download?

Generally, seismologist uses the [SEED](https://ds.iris.edu/ds/nodes/dmc/data/formats/seed/) format which has 4 components for data identification:
1) Network code
2) Station code
3) Location ID
4) Channel codes

We can check global seismic station distribution and get the identification information from the IRIS [GMAP](https://ds.iris.edu/gmap/) tool which creates dynamic station maps that can be panned and zoomed. In this example, I retrieved network code, station code, channel codes and the location ID from the GMAP tool. The starttime and endtime depend on when the events occured.

In [ ]:
network = 'II'
station = 'KDAK'
channel = 'BH?'
location = '00'
starttime = '2021-07-28'
endtime = '2021-07-30'

Now let's query the station service of the FDSN client using the [get_stations](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.client.Client.get_stations.html) function which returns a stations inventory object.

In [ ]:
inventory = client.get_stations(network=network, station=station, channel=channel, 
                                level='response', location=location, starttime=UTCDateTime(starttime), 
                                endtime=UTCDateTime(endtime))

In [ ]:
print(inventory)

In [ ]:
print(inventory[0])
print(inventory[0].code)
print(inventory[0].stations)
print(inventory[0][0])
print(inventory[0][0].code) # For station name
print(inventory[0][0][0])

---
We can view the stations on a map global or local map using the [inventory.plot()](https://docs.obspy.org/master/packages/autogen/obspy.core.inventory.inventory.Inventory.plot.html) function. See the documentation for more parameters.

NOTE: Plotting higher resolution maps i.e. resolution='f' will take longer to plot.

In [ ]:
inventory.plot('global');
#inventory.plot('ortho');
inventory.plot(projection='local', resolution='f', color='blue', continent_fill_color='peru', water_fill_color='lightblue');

Now we create a new directory, name it appropriately and save the station inventory xml file inside it.

First we import python's [os](https://docs.python.org/3/library/os.html) module which provides a portable of interfacing with the computer's operation system.

In [ ]:
import os

Use the os.path.join() method to establish a path to the directory we want to create.

In [ ]:
data_dir = os.path.join('data', '')
print(data_dir)

---
Use the os.mkdir() method to create a directory joined by the path

In [ ]:
os.mkdir(data_dir)

Now save the station inventory inside the newly created directory using ObsPy's [inventory.write()](https://docs.obspy.org/master/packages/autogen/obspy.core.inventory.inventory.Inventory.write.html) function.

In [ ]:
inventory.write(data_dir + 'station.xml', 'STATIONXML')

---
To read station inventory from an xml file in your computer, import ObsPy's [inventory.read()](https://docs.obspy.org/packages/autogen/obspy.core.inventory.inventory.read_inventory.html) function.

In [ ]:
from obspy import read_inventory

Read the inventory into a new inventory variable.

In [ ]:
inventory2 = read_inventory(data_dir + 'station.xml')

In [ ]:
print(inventory2)

In [ ]:
inventory2.plot();

## 2. Downloading earthquake catalog

For this example, let's create a catalog for the 2021 [M8.2 Chignik earthquake](https://earthquake.usgs.gov/earthquakes/eventpage/ak0219neiszm/executive).

We will begin by querying the events service of the FDSN client using the [get_events](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.client.Client.get_events.html) function which returns an events catalog object.

In [ ]:
client = FDSN_Client("IRIS")

In [ ]:
catalog = client.get_events(starttime='2021-07-28', endtime='2021-07-30', minmagnitude=8)

In [ ]:
print(catalog)

In [ ]:
catalog.plot();

Write event to file

In [ ]:
catalog.write(data_dir + 'event.xml', 'QUAKEML')

To read event(s) from file, import ObsPy's [read_events()](https://docs.obspy.org/master/packages/autogen/obspy.core.event.read_events.html) function.

In [ ]:
from obspy import read_events

In [ ]:
catalog2 = read_events(data_dir + 'event.xml')

In [ ]:
print(catalog2)

In [ ]:
catalog2.plot();

In [ ]:
print(catalog2[0])

---
Explore earthquake parameters from the catalog:

In [ ]:
event = catalog2[0]
eq_time = event.origins[0].time
eq_mag = event.magnitudes[0].mag
eq_lat = event.origins[0].latitude
eq_lon = event.origins[0].longitude
eq_depth = event.origins[0].depth/1000
eq_description = event.event_descriptions[0].text
eq_mag_type = event.magnitudes[0].magnitude_type

In [ ]:
print(eq_time)

## 3. Downloading earthquake seismograms

Lets download seismograms for the 2021 [M8.2 Chignik earthquake](https://earthquake.usgs.gov/earthquakes/eventpage/ak0219neiszm/executive) recorded at the nearby II.KDAK station.

First we import ObsPy's [geodetics](https://docs.obspy.org/master/packages/obspy.geodetics.html) and [taup](https://docs.obspy.org/packages/obspy.taup.html) functions.

In [ ]:
from obspy.clients.fdsn import Client as FDSN_Client
from obspy import UTCDateTime
from obspy.taup import TauPyModel
from obspy.geodetics import gps2dist_azimuth
from obspy.geodetics import kilometer2degrees

Define the FDSN client and taup earth model

In [ ]:
client = FDSN_Client("IRIS")
tpmodel = TauPyModel(model='iasp91')

Get the station latitude and longitude from the station inventory we downloaded earlier.

In [ ]:
sta_lat = inventory[0].stations[0].latitude
sta_lon = inventory[0].stations[0].longitude

Retrieve the earthquakes P-wave travel time information.

In [ ]:
epi_dist_m, az, baz = gps2dist_azimuth(eq_lat, eq_lon, sta_lat, sta_lon)
epi_dist_km = epi_dist_m/1000
epi_dist_deg = kilometer2degrees(epi_dist_km)
arrivals = tpmodel.get_travel_times(distance_in_degree=epi_dist_deg, source_depth_in_km=eq_depth, phase_list=['P'])
print(arrivals[0])

---
Let's see if we have all the information we need to download the seismograms.

In [ ]:
ttime = arrivals[0].time
print(ttime)

In [ ]:
print(eq_time)

In [ ]:
#originTime = UTCDateTime('2021-07-29T06:15:49')
#print(originTime)

In [ ]:
onset = eq_time + ttime
print(onset)

In [ ]:
tstart = onset - 60*3
print(tstart)

In [ ]:
tend = onset + 60*15
print(tend)

In [ ]:
network = 'II'#inventory[0].code
station = 'KDAK'#inventory[0].stations[0].code
location = '00'#'*'
channel = 'BH?'

---
Query the dataselect service of the FDSN client using the [get_waveforms](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.client.Client.get_waveforms.html) function which returns a [Stream](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.html#obspy.core.stream.Stream) object.

ObsPy stores seismograms in [Stream](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.html#obspy.core.stream.Stream) object which are list-like objects that contain several [Trace](https://docs.obspy.org/packages/autogen/obspy.core.trace.Trace.html#obspy.core.trace.Trace) objects (i.e. time series with accompanying header/metadata).

The data attribute in Trace objects contains the actual time series data which is a [NumPy](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html#numpy.ndarray) array. The metadata is stored in [stats]() attribute of the Trace objects.

In [ ]:
stream = client.get_waveforms(network, station, location, channel, tstart, tend)
print(stream)

In [ ]:
len(stream)

Assign the first trace to a variable

In [ ]:
print(stream[0])

In [ ]:
tr = stream[0].copy()
print(tr)

Access meta data using the stats keyword.

In [ ]:
print(tr.stats)

In [ ]:
print(tr.stats.station)

In [ ]:
print(tr.stats.mseed.record_length)

Access waveform data using the data keyword.

In [ ]:
len(tr)

In [ ]:
tr.data

In [ ]:
tr.data[0:3]

Use the [plot](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.plot.html#obspy.core.stream.Stream.plot) method to plot the waveform.

In [ ]:
tr.plot();

In [ ]:
print(inventory.get_orientation("II.KDAK.00.BHZ", eq_time))
print(inventory.get_response("II.KDAK.00.BHZ", eq_time))
print(inventory.get_coordinates("II.KDAK.00.BHZ", eq_time))
print(inventory.get_channel_metadata("II.KDAK.00.BHZ", eq_time))

---
## 4. Removing instrument response

The digitizer in many broadband seismometers converts the analog voltage signal to a digital timeseries. So the seismogram we downloaded is not actually a record ground motion but counts (see the cell above). To convert the data to actual ground motion (acceleration, velocity or displacement), we have to remove the instrument response.

ObsPy allows users to attach the instrument response from clients when waveforms are downloaded.

In [ ]:
channel = 'BH1'
stream = client.get_waveforms(network, station, location, channel, tstart, tend, attach_response=True)
print(stream)
stream.plot();

In [ ]:
inst_removed = stream.copy()
print(inst_removed)

In [ ]:
inst_removed[0].remove_response(output='VEL')

stream[0].plot();
inst_removed[0].plot(color='green');

We can view the instrument removal process making the plot parameter True.

Let's repeat the instrument removal. We have to re-copy the data since the instrument response was already removed.

In [ ]:
inst_removed = stream.copy()
inst_removed.remove_response(output='VEL', plot=True)

## 5. Writing waveform data to disk

In [ ]:
JDay = str(eq_time.julday)
YYYY = str(eq_time.year)
MM = str((format(eq_time.month,'02d')))
DD = str((format(eq_time.day,'02d')))
HH = str((format(eq_time.hour,'02d')))
mm = str((format(eq_time.minute,'02d')))
ss = str((format(eq_time.second,'02d')))
msec = str(eq_time.microsecond)[:6]

In [ ]:
location = inst_removed[0].stats.location
quality = inst_removed[0].stats.mseed.get('dataquality')
fileID = network+'.'+station+'.'+location+'.'+channel+'.'+ quality+'.'+YYYY+'.'+JDay+'.'+HH+mm+ss+'.SAC'
print(fileID)

In [ ]:
inst_removed[0].write(data_dir + fileID, format='SAC')

## 6. Reading waveform data from disk

ObsPy can detect seismogram data of various formats (e.g. SAC, MiniSEED, GSE2, SEISAN, Q, etc.) and import them into a Stream object using the [read()](https://docs.obspy.org/packages/autogen/obspy.core.stream.read.html) function.

In [ ]:
from obspy import read

In [ ]:
print(data_dir+fileID)

In [ ]:
st = read(data_dir + fileID)
print(st)

In [ ]:
print(st[0].stats) # Notice the file format has changed from mseed to sac.

In [ ]:
st[0].plot();

## 7. Miscellaneous

---
### Trimming

In [ ]:
tstart2 = onset - 20
tend2 = onset + 3*60

In [ ]:
st_trimmed = st.copy()
st_trimmed.trim(starttime=tstart2, endtime=tend2)

In [ ]:
st_trimmed.plot();

---
### Spectrogram

A spectrogram is a visual way of representing the strength of a signal over time at different frequencies.

ObsPy's [spectrogram](https://docs.obspy.org/packages/autogen/obspy.imaging.spectrogram.spectrogram.html) function computes and plots spectrograms of input waveforms.

In [ ]:
st_trimmed.spectrogram(log=True, title='II.KDAK   ' + str(st_trimmed[0].stats.starttime))

---
### Resampling

ObsPy provides several methods for changing the sampling rate of waveform data. In this example, we will use the [interpolate](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.interpolate.html#obspy.core.stream.Stream.interpolate) function.

Since this operation alters the actual data arrays, keep the original data using the copy function to create a copy of your stream object.

Print original sampling rate

In [ ]:
st_resamp = st.copy()
print(st_resamp[0].stats.sampling_rate)

Now let's resample to 10 Hz.

In [ ]:
st_resamp.interpolate(10)

Print the new sampling rate

In [ ]:
print(st_resamp[0].stats.sampling_rate)

In [ ]:
st_resamp.plot();

---
### Filtering

Available filters in ObsPy include:
1) bandpass
2) bandstop
3) lowpass
4) highpass
    
In this example we will apply a bandpass filter to the resampled chignik seismogram.

In [ ]:
st_filt = st_resamp.copy()
st_filt.filter('bandpass', freqmin=0.01, freqmax=0.1, corners=4, zerophase=True)
st_resamp.plot();
st_filt.plot();

<h1 style="text-align:center; color:#3333ff;">That's all folks!</h1>